In [18]:
import streamlit as st
import os
import pandas as pd
import random
from os.path import join
from datetime import datetime
# from src import decorate_with_code, show_response, get_from_user
from dotenv import load_dotenv
from langchain_groq.chat_models import ChatGroq
from langchain_mistralai import ChatMistralAI
from langchain_google_genai import GoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from huggingface_hub import HfApi
load_dotenv()

import pandas as pd

In [19]:
prompts = pd.read_csv("prompts.csv")
prompts.head(2)

,prompt,Desired Answer,Category,llama3_answer_1721726247,llama3_score_1721726247,mixtral_answer_1721726407,mixtral_score_1721726407,gemma_answer_1721726499,gemma_score_1721726499,Codestral Mamba_answer_1721759526,Codestral Mamba_score_1721759526,Codestral_answer_1721759762,Codestral_score_1721759762
0,Which month has the highest average PM2.5 in 2...,January,NaN,The month with the highest average PM2.5 in 20...,True,The month with the highest average PM2.5 in 20...,True,The highest average PM2.5 in 2023 for Mumbai w...,True,content='To find the month with the highest av...,True,The month with the highest average PM2.5 in 20...,True
1,Which month generally has the highest pollution?,November,NaN,The month with the highest pollution is 11 wit...,True,The month with the highest pollution (on avera...,True,The month with the highest average PM2.5 is No...,True,content='To find out which month generally has...,False,The month with the highest pollution is Novemb...,True


In [28]:
data = pd.read_parquet("AQI_data.parquet")
print(data.dtypes)
data.head(2)

Date                                      datetime64[ns]
City                                              object
AQI                                              float64
Pollutant                                         object
Air Quality                                       object
Based on number of monitoring stations           float64
State                                             object
Date_City                                         object
dtype: object


,Date,City,AQI,Pollutant,Air Quality,Based on number of monitoring stations,State,Date_City
0,2016-01-01,Agartala,NaN,None,None,NaN,None,None
1,2016-01-01,Agra,417.0,PM\n2.5,Severe,1.0,Uttar Pradesh,2016-01-01_Agra


In [34]:
def get_full_prompt(question):
    return f"""You are a data scientist who is good with tools such as pandas, numpy and matplotlib. You are also an expert in air quality. You
can access daily AQI `data` and you have to complete a code provided by me based on my question. `data` is a pandas DataFrame and has the following columns and data types:

Date: date, Date of the `AQI` data
City: string, Name of the city where the `AQI` was recorded
State: string, Name of the state where `City` is located
AQI: float, AQI value
Air Quality: string, Air quality category from ["Satisfactory", "Moderate", "Good", "Poor", "Very Poor", "Severe"] based on `AQI` value

Now, my question is: "{question}"

Complete the code below to answer my question:

```python
import pandas as pd
import numpy as np

data = pd.read_parquet("AQI_data.parquet")"""

In [35]:
print(get_full_prompt(prompts.prompt[0]))

You are a data scientist who is good with tools such as pandas, numpy and matplotlib. You are also an expert in air quality. You
can access daily AQI `data` and you have to complete a code provided by me based on my question. `data` is a pandas DataFrame and has the following columns and data types:

Date: date, Date of the `AQI` data
City: string, Name of the city where the `AQI` was recorded
State: string, Name of the state where `City` is located
AQI: float, AQI value
Air Quality: string, Air quality category from ["Satisfactory", "Moderate", "Good", "Poor", "Very Poor", "Severe"] based on `AQI` value

Now, my question is: "Which month has the highest average PM2.5 in 2023 for Mumbai?"

Complete the code below to answer my question:

```python
import pandas as pd
import numpy as np

data = pd.read_parquet("AQI_data.parquet")


In [74]:
def get_gemini_response(prompt, model):
    return GoogleGenerativeAI(model=model, google_api_key=os.environ.get("GOOGLE_API_KEY"), temperature=0).invoke(prompt)

def get_groq_response(prompt, model):
    return ChatGroq(model=model, api_key=os.environ.get("GROQ_API_KEY"), temperature=0).invoke(prompt).content

llms = {"gemini-pro": lambda prompt: get_gemini_response(prompt, "gemini-pro"), "groq_gemma-7b-it": lambda prompt: get_groq_response(prompt, "gemma-7b-it"), "groq_llama-3.2-90b-text-preview": lambda prompt: get_groq_response(prompt, "llama-3.2-90b-text-preview")}

In [75]:
i = 1
prompts.prompt[i], prompts['Desired Answer'][i]

('Which month generally has the highest pollution?', 'November')

In [ ]:
llm = "groq_llama-3.2-90b-text-preview"
full_prompt = get_full_prompt("Which city has the highest AQI value consistently over the years?")
answer = llms[llm](full_prompt)

In [83]:
import re
code = re.search(r"```python\n(.*)```", answer, re.DOTALL).group(1)
print(full_prompt)
print("#" * 100)
print(code)
print("#" * 100)
exec(code)

You are a data scientist who is good with tools such as pandas, numpy and matplotlib. You are also an expert in air quality. You
can access daily AQI `data` and you have to complete a code provided by me based on my question. `data` is a pandas DataFrame and has the following columns and data types:

Date: date, Date of the `AQI` data
City: string, Name of the city where the `AQI` was recorded
State: string, Name of the state where `City` is located
AQI: float, AQI value
Air Quality: string, Air quality category from ["Satisfactory", "Moderate", "Good", "Poor", "Very Poor", "Severe"] based on `AQI` value

Now, my question is: "Which city has the highest AQI value consistently over the years?"

Complete the code below to answer my question:

```python
import pandas as pd
import numpy as np

data = pd.read_parquet("AQI_data.parquet")
####################################################################################################
import pandas as pd
import numpy as np
import matplotli

In [ ]:
# import streamlit as st
# import os
# import pandas as pd
# import random
# from os.path import join
# from datetime import datetime
# from src import decorate_with_code, show_response, get_from_user
# from dotenv import load_dotenv
# from langchain_groq.chat_models import ChatGroq
# from langchain_mistralai import ChatMistralAI
# from huggingface_hub import HfApi
# st.set_page_config(layout="wide")

# ### Extract data.zip
# if not os.path.exists("data/1"):
#     os.system("unzip data.zip")

# # Load environment variables : Groq and Hugging Face API keys
# load_dotenv()
# Groq_Token = os.environ["GROQ_API_KEY"]
# CODESTRAL_API_KEY = os.environ["CODESTRAL_API_KEY"]
# hf_token = os.environ["HF_TOKEN"]
# models = {"llama3":"llama3-70b-8192","mixtral": "mixtral-8x7b-32768", "llama2": "llama2-70b-4096", "gemma": "gemma-7b-it"}
# groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}
# mistral_models = {"Codestral Mamba" : "open-codestral-mamba", "Codestral" : "codestral-latest","Mistral 7B":"open-mistral-7b"}
# groq_model_list = list(groq_models.keys())
# mistral_model_list = list(mistral_models.keys())

# self_path = os.path.dirname(os.path.abspath(__file__))


# def generate_template(prompt):
#     df_check = pd.read_csv("Data.csv")
#     df_check["Timestamp"] = pd.to_datetime(df_check["Timestamp"])
#     df_check = df_check.head(5)

#     new_line = "\n"

#     template = f"""```python
# import pandas as pd
# import matplotlib.pyplot as plt

# df = pd.read_csv("Data.csv")
# df["Timestamp"] = pd.to_datetime(df["Timestamp"])

# # df.dtypes
# {new_line.join(map(lambda x: '# '+x, str(df_check.dtypes).split(new_line)))}

# # {prompt.strip()}
# # <your code here>

# #answer = 
# ```
# """
#     return template


# def generate_query(template):
    
#     query = f"""I have a pandas dataframe data of PM2.5.
# * The columns are 'Timestamp', 'station', 'PM2.5', 'address', 'city', 'latitude', 'longitude', and 'state'.
# * Frequency of Data is Daily.
# * `Pollution` generally means `PM2.5`.
# * PM2.5 guidelines: India: 60, WHO: 15.
# * Store the final answer in a global variable `answer`.
# * Always report the unit of the data. Example: `The average PM2.5 is 45.67 µg/m³`

# Complete the following code.

# {template}
# """
#     return query


# def process_query(query, llm):
#     global answer
#     template = generate_template(query)
#     query = generate_query(template)
#     global code
#     global error
#     try:
#         answer = llm.invoke(query)
#         error = ''
#         code = f"""
#         {template.split("```python")[1].split("```")[0]}
#         {answer.content.split("```python")[1].split("```")[0]}
#         """
#         # update variable `answer` when code is executed
#         exec(code,globals())
#     except Exception as e:
#         error = e
#         code = ''
#         answer = f"Error: {e}"
#     print(answer)


# # Using HTML and CSS to center the title
# st.write(
#     """
#     <style>
#     .title {
#         text-align: center;
#         color: #17becf;
#     }
#     </style>
# """,
#     unsafe_allow_html=True,
# )




#     # Display images and text in three columns with specified ratios
# col1, col2, col3 = st.sidebar.columns((1.0, 2, 1.0))  
# with col2:
#     st.markdown("<h1 class='title'>Airchat</h1>", unsafe_allow_html=True)
    
    
# model_name = st.sidebar.selectbox("Select LLM:", groq_model_list + mistral_model_list)

# questions = ['Custom Prompt']
# with open(join(self_path, "questions.txt")) as f:
#     questions += f.read().split("\n")

# waiting_lines = ("Thinking...", "Just a moment...", "Let me think...", "Working on it...", "Processing...", "Hold on...", "One moment...", "On it...")



# # Initialize chat history
# if "responses" not in st.session_state:
#     st.session_state.responses = []
  

# # Display chat responses from history on app rerun
# print("#"*10)
# for response_id, response in enumerate(st.session_state.responses):
#     status = show_response(st, response)
#     if response["role"] == "assistant":
#         # feedback_key = f"feedback_{int(response_id/2)}"
#         print("response_id", response_id)
        
#         error = response["error"]
#         output = response["content"]
#         last_prompt = response["last_prompt"]
#         code = response["gen_code"]
#         evaluation = response["evaluation"]
        
                
        
# print("#"*10)

# show = True
# prompt = st.sidebar.selectbox("Select a Prompt:", questions, key="prompt_key")
# if prompt == 'Custom Prompt':
#     show = False
#     # React to user input
#     prompt = st.chat_input("Ask me anything about air quality!", key=1000)
#     if prompt :
#         show = True
# else:
#     # placeholder for chat input
#     st.chat_input("Select 'Select a Prompt' -> 'Custom Prompt' in the sidebar to ask your own questions.", key=1000, disabled=True)

# if "last_prompt" in st.session_state:
#     last_prompt = st.session_state["last_prompt"]
#     last_model_name = st.session_state["last_model_name"]
#     if (prompt == last_prompt) and (model_name == last_model_name):
#         show = False

# if prompt:
#     st.sidebar.info("Select 'Custom Prompt' to ask your own questions.")

#     if show:
#         # Add user input to chat history
#         user_response = get_from_user(prompt)
#         st.session_state.responses.append(user_response)

#         # select random waiting line
#         with st.spinner(random.choice(waiting_lines)):
#             ran = False
#             for i in range(1):
#                 print(f"Attempt {i+1}")
#                 if model_name in groq_models:
#                     model_folder = "Groq_" + groq_models[model_name]
#                     llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
#                 else:
#                     model_folder = "MistralAI_" + mistral_models[model_name]
#                     llm = ChatMistralAI(model=mistral_models[model_name], api_key=CODESTRAL_API_KEY, temperature=0)
#                 print(llm)
#                 # llm = ChatGroq(model=models[model_name], api_key=os.getenv("GROQ_API"), temperature=0)

#                 df_check = pd.read_csv("Data.csv")
#                 df_check["Timestamp"] = pd.to_datetime(df_check["Timestamp"])
#                 df_check = df_check.head(5)

#                 new_line = "\n"

#                 parameters = {"font.size": 12,"figure.dpi": 600}

#                 process_query(prompt, llm)
                
                
#                 # Read the questions from Questions.txt and find the index of the question if there is a match
#                 with open(join("questions.txt")) as f:
#                     questions = f.read().split("\n")
#                     try:
#                         index = questions.index(prompt)
#                         index = index + 1
#                     except:
#                         index = None 
#                 print("Index",index)
#                 if type(index) == int:
#                     # Open folder data/index/llm_name and compare with evaluation.txt
#                     with open(join("data", str(index), model_folder, "evaluation.txt")) as f:
#                         evaluation = f.read().strip()
#                     with open(join("data", str(index), "ground_truth", "answer.txt")) as f:
#                         ground_truth = f.read().strip()
#                 else:
#                     evaluation = "DK"
#                     ground_truth = None 
#                 response = {"role": "assistant", "content": answer, "gen_code": code, "ex_code": code, "last_prompt": prompt, "error": error,"evaluation": evaluation,"ground_truth": ground_truth}

#                 if ran:
#                     break
            
#         # Append agent response to chat history
#         st.session_state.responses.append(response)
        
#         st.session_state['last_prompt'] = prompt
#         st.session_state['last_model_name'] = model_name
#         st.rerun()
        
        

# # Display contact details with message
# st.sidebar.markdown("<hr>", unsafe_allow_html=True)